<a href="https://colab.research.google.com/github/tamoghna21/RAG_LLM/blob/main/1c_RAG_QA_pdf_full_with_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Full Implementation of RAG framework to get answer from private pdf documents using LLM (Mistral-7B-Instruct-v0.1)

### Retrieval-Augmented generation on local pdf documents (Federal Open Market Committee (FOMC) meeting documents for the years 2020-2023)

#### Select Runtime > GPU

#### Install Packages

In [ ]:
!pip install -q torch transformers accelerate bitsandbytes langchain sentence-transformers faiss-gpu
#!pip install -q ragatouille
!pip install -q langchain_community
!pip install -q python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.9 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document as LangchainDocument
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from transformers import AutoTokenizer, pipeline
#from ragatouille import RAGPretrainedModel
from transformers import Pipeline
from typing import Optional, List, Tuple
#import pytesseract
#from PIL import ImageEnhance, ImageFilter, Image


#### Path of the Vector database (already created from the pdf docs)

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

os.chdir("/content/drive/My Drive/")

from dotenv import load_dotenv
load_dotenv(os.path.join('', './.env'))
os.environ["HUGGINGFACE_TOKEN"] = os.getenv('HUGGINGFACE_TOKEN')

# Folder where the FAISS Index is stored
os.chdir("/content/drive/My Drive/FOMC_docs_2023_2020")

Mounted at /content/drive


#### Load the Vector database, load the LLM model, setup the prompt for the LLM model

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
#from langchain.chains import LLMChain

EMBEDDING_MODEL_NAME = "thenlper/gte-small"
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)
db_VECTOR = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)

from huggingface_hub import login
login(token=os.environ["HUGGINGFACE_TOKEN"])

READER_MODEL_NAME = 'mistralai/Mistral-7B-Instruct-v0.1' # The LLM Model

tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

use_4bit = True # Activate 4-bit precision base model loading
compute_dtype = getattr(torch, "float16") # Compute dtype for 4-bit base models
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit, # Activate 4-bit precision base model loading
    bnb_4bit_use_double_quant=False, #True, # Activate nested quantization for 4-bit base models (double quantization)
    bnb_4bit_quant_type="nf4", # Quantization type (fp4 or nf4)
    bnb_4bit_compute_dtype=compute_dtype #torch.bfloat16
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME,quantization_config=bnb_config)


READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text = False,
    max_new_tokens=1000,
)

langchain_llm = HuggingFacePipeline(pipeline=READER_LLM)

# Create prompt template
prompt_template = """
### [INST] Instruction: Answer the question based on your knowledge. Here is context to help:

{context}

### QUESTION:
{question} [/INST]
"""

# Create prompt from prompt template
prompt = PromptTemplate(
  input_variables=["context", "question"],
  template=prompt_template,
)

# Create llm chain
llm_chain = prompt | langchain_llm

retriever = db_VECTOR.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 5})

rag_chain = (
  {"context": retriever, "question": RunnablePassthrough()}
      | llm_chain
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


## Ask questions

In [ ]:
question = "How is the inflation trend in 2023?"

# Asking without RAG
output = llm_chain.invoke({"context":"",
                  "question": question})
print(output)


I don't have real-time data or the ability to browse the internet, so I can't provide the most up-to-date information about the inflation trend in 2023. However, as of my last update, the global inflation rate has been increasing over the past few years due to various factors such as supply chain disruptions, energy price increases, and labor shortages. Many countries have experienced higher inflation rates than their historical averages. However, the specific inflation trend for 2023 would depend on how these and other factors evolve in the coming months. It's always a good idea to check reliable sources like central bank websites or financial news outlets for the latest information.


In [ ]:
#Asking RAG supported LLM
output = rag_chain.invoke(question)
print(output)



The information provided does not give a specific answer on how the inflation trend will be in 2023. It discusses the views of participants regarding the inflation trend in 2023 and how they expect it to change. According to the document, participants expect that inflation will decline significantly over the course of 2022 as supply constraints ease, but they have revised up their forecasts of inflation for 2022 notably, and many did so for 2023 as well. They point to rising housing costs and rents, more widespread wage growth driven by labor shortages, and more prolonged global supply-side frictions as potential factors that could exacerbate inflation.


In [ ]:
question = "What is the set federeral fund rate in February 2023?"
output = llm_chain.invoke({"context":"",
                  "question": question})
print(output)


As of my knowledge up to February 2023, the federal funds rate is not publicly available as it is determined by the Federal Reserve and can change frequently. You can check the official website of the Federal Reserve or contact them directly for the most accurate and up-to-date information.


In [ ]:
output = rag_chain.invoke(question)
print(output)


It is not possible to determine the exact federal funds rate for February 2023 based solely on the provided information. The document mentions that the projected appropriate policy path for the federal funds rate in February 2023 is between 2.0 and 2.2, but it does not provide the actual rate for that month.


#### References:
https://huggingface.co/learn/cookbook/en/advanced_rag

https://medium.com/@akriti.upadhyay/implementing-rag-with-langchain-and-hugging-face-28e3ea66c5f7

https://medium.com/@s.rashwand/how-to-build-a-chatbot-smarter-than-chatgpt-quickly-using-langchain-and-weaviate-f6309cc86e09

https://medium.com/@thakermadhav/build-your-own-rag-with-mistral-7b-and-langchain-97d0c92fa146